In [10]:
import numpy as np 
import pandas as pd 
import matplotlib .pyplot as plt
import seaborn as sns

In [11]:
X = pd.read_excel(r"C:\Users\wasim\Office\Machine Learning Projects\Flight Fare Prediction\Data_Train.xlsx")
pd.set_option( "display.max_columns ", None)
X.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [12]:
X["Journey_day"] = pd.to_datetime(X.Date_of_Journey, format="%d/%m/%Y").dt.day
X["Journey_month"] = pd.to_datetime(X["Date_of_Journey"], format = "%d/%m/%Y").dt.month

X.drop(["Date_of_Journey"], axis = 1, inplace = True)


In [13]:
# Extracting Hours
X["Dep_hour"] = pd.to_datetime(X["Dep_Time"]).dt.hour

# Extracting Minutes
X["Dep_min"] = pd.to_datetime(X["Dep_Time"]).dt.minute

# Now we can drop Dep_Time as it is of no use
X.drop(["Dep_Time"], axis = 1, inplace = True)

X["Arrival_hour"] = pd.to_datetime(X.Arrival_Time).dt.hour

# Extracting Minutes
X["Arrival_min"] = pd.to_datetime(X.Arrival_Time).dt.minute

# Now we can drop Arrival_Time as it is of no use
X.drop(["Arrival_Time"], axis = 1, inplace = True)

In [14]:
# Assigning and converting Duration column into list
duration = list(X["Duration"])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:    # Check if duration contains only hour or mins
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
        else:
            duration[i] = "0h " + duration[i]           # Adds 0 hour

duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))    # Extract hours from duration
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))   # Extracts only 

In [15]:
X["Duration_hours"] = duration_hours
X["Duration_mins"] = duration_mins

X.drop(["Duration"], axis = 1, inplace = True)

In [16]:
X.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Journey_day,Journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,9,6,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,12,5,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,1,3,16,50,21,35,4,45


In [17]:
X.drop(["Route", "Additional_Info"], axis = 1, inplace = True)


In [18]:
X.head()

,Airline,Source,Destination,Total_Stops,Price,Journey_day,Journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins
0,IndiGo,Banglore,New Delhi,non-stop,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,2 stops,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,2 stops,13882,9,6,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,1 stop,6218,12,5,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,1 stop,13302,1,3,16,50,21,35,4,45


In [19]:
X.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)


In [20]:
X.head()

,Airline,Source,Destination,Total_Stops,Price,Journey_day,Journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins
0,IndiGo,Banglore,New Delhi,0.0,3897,24,3,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,2.0,7662,1,5,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,2.0,13882,9,6,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,1.0,6218,12,5,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,1.0,13302,1,3,16,50,21,35,4,45


In [21]:
def transform_x(X):
    X["Journey_day"] = pd.to_datetime(X.Date_of_Journey, format="%d/%m/%Y").dt.day
    X["Journey_month"] = pd.to_datetime(X["Date_of_Journey"], format = "%d/%m/%Y").dt.month

    X.drop(["Date_of_Journey"], axis = 1, inplace = True)

    # Extracting Hours
    X["Dep_hour"] = pd.to_datetime(X["Dep_Time"]).dt.hour

    # Extracting Minutes
    X["Dep_min"] = pd.to_datetime(X["Dep_Time"]).dt.minute

    # Now we can drop Dep_Time as it is of no use
    X.drop(["Dep_Time"], axis = 1, inplace = True)

    X["Arrival_hour"] = pd.to_datetime(X.Arrival_Time).dt.hour

    # Extracting Minutes
    X["Arrival_min"] = pd.to_datetime(X.Arrival_Time).dt.minute

    # Now we can drop Arrival_Time as it is of no use
    X.drop(["Arrival_Time"], axis = 1, inplace = True)

    # Assigning and converting Duration column into list
    duration = list(X["Duration"])

    for i in range(len(duration)):
        if len(duration[i].split()) != 2:    # Check if duration contains only hour or mins
            if "h" in duration[i]:
                duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
            else:
                duration[i] = "0h " + duration[i]           # Adds 0 hour

    duration_hours = []
    duration_mins = []
    for i in range(len(duration)):
        duration_hours.append(int(duration[i].split(sep = "h")[0]))    # Extract hours from duration
        duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))   # Extracts only 

    X["Duration_hours"] = duration_hours
    X["Duration_mins"] = duration_mins

    X.drop(["Duration"], axis = 1, inplace = True)

    X.drop(["Route", "Additional_Info"], axis = 1, inplace = True)

    X.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)

    return X

In [72]:
Airline =  "Jet Airways "
Date_of_Journey =  "9/06/2019 "
Source =  "Banglore "
Destination =  "Cochin "
Dep_Time =  "09:25 "
Arrival_Time =  "04:25 "
Duration =  "19h "
Total_Stops = 2

[ "Airline ",  "Source ",  "Destination ",  "Total_Stops ",
        "Journey_day ",  "Journey_month ",  "Dep_hour ",  "Dep_min ",  "Arrival_hour ",
        "Arrival_min ",  "Duration_hours ",  "Duration_mins "]
flight_data_dict = {
                "Airline":  [Airline],
                "Date_of_Journey":  [Date_of_Journey],
                "Source":  [Source],
                "Destination":  [Destination],
                "Dep_Time":  [Dep_Time],
                "Arrival_Time":  [Arrival_Time],
                "Duration":  [Duration],
                "Total_Stops":  [Total_Stops]
                }
df = pd.DataFrame.from_dict(flight_data_dict)
journey_date = df.Date_of_Journey.iloc[0].split( "/ ")

df[ "Journey_day "] = journey_date[0]
df[ "Journey_month "] = journey_date[1]

Dep_Time = df.Dep_Time.iloc[0].split( ": ")
df[ "Dep_hour "] = Dep_Time[0]
df[ "Dep_min "] = Dep_Time[1]

Arrival_Time = df.Arrival_Time.iloc[0].split( ": ")
df[ "Arrival_hour "] = Arrival_Time[0]
df[ "Arrival_min "] = Arrival_Time[1]

duration = df.Duration.iloc[0]
if len(duration.split()) != 2:    # Check if duration contains only hour or mins
    if "h" in duration:
        duration = duration.strip() + " 0m"   # Adds 0 minute
    else:
        duration = "0h " + duration           # Adds 0 hour

df[ "Duration_hours "] = int(duration.split(sep = "h")[0])
df[ "Duration_mins "] = int(duration.split(sep = "m")[0].split()[-1])

df.drop(["Date_of_Journey",  "Dep_Time ",  "Arrival_Time ",  "Duration "], axis = 1, inplace = True)

df

,Airline,Source,Destination,Total_Stops,Journey_day,Journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins
0,Jet Airways,Banglore,Cochin,2,9,06,09,25,04,25,19,0


In [22]:
X = pd.read_excel(r"C:\Users\wasim\Office\Machine Learning Projects\Flight Fare Prediction\Data_Train.xlsx")

data = X.iloc[0]

X = transform_x(data)

{
    "Airline" :  "Jet Airways",
     "Date_of_Journey " :  "9/06/2019",
     "Source " :  "Banglore",
     "Destination " :  "Cochin",
     "Dep_Time" :  "09:25",
     "Arrival_Time" :  "04:25",
     "Duration" :  "19h",
     "Total_Stops" : 2,
}

AttributeError: 'Timestamp' object has no attribute 'dt'

In [ ]:
pd.read_excel(r "C:\Users\wasim\Office\Machine Learning Projects\Flight Fare Prediction\flightFare\artifact\data_transformation\2022-09-05-22-26-17\transformed_data\train\Data_train.xlsx ")

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [23]:
import pandas as pd
import numpy as np

npz = np.load(r "C:\Users\wasim\Office\Machine Learning Projects\Flight Fare Prediction\flightFare\artifact\data_transformation\2022-09-10-00-38-22\transformed_data\train\Data_train.npz ")
npz

array([[ 2.62912003e-01,  8.73536039e-01, -1.47302050e+00, ...,
         0.00000000e+00,  0.00000000e+00,  7.69500000e+03],
       [ 2.62912003e-01, -8.96540545e-01, -6.12743512e-01, ...,
         0.00000000e+00,  0.00000000e+00,  5.40600000e+03],
       [-1.21819938e+00,  1.22755136e+00, -6.12743512e-01, ...,
         0.00000000e+00,  0.00000000e+00,  4.82300000e+03],
       ...,
       [ 2.62912003e-01, -5.42525228e-01, -1.47302050e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.40860000e+04],
       [ 2.62912003e-01, -5.42525228e-01,  1.10781047e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.05770000e+04],
       [-1.21819938e+00,  1.22755136e+00, -6.12743512e-01, ...,
         0.00000000e+00,  0.00000000e+00,  4.82300000e+03]])

In [24]:
npz.shape

(8546, 32)

In [44]:
columns = [ "Airline ",  "Source ",  "Destination ",  "Total_Stops ",
        "Journey_day ",  "Journey_month ",  "Dep_hour ",  "Dep_min ",  "Arrival_hour ",
        "Arrival_min ",  "Duration_hours ",  "Duration_mins "]
values = [ "Jet Airways ",  "Delhi ",  "Cochin ", 1,  24, 6, 13, 0, 4, 25, 15, 25]
# Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882

"""
Airline - No change
Source - No change
Destination - No change
Total_stops - Replace 
journey_day - Date_of_journey
journey_month - Date_of_journey
       - Dep_Time
       - Dep_Time
       - Arrival_Time
       - Arrival_Time
       - Duration
       - Duration

"""

print(values)

df = pd.DataFrame(columns=columns)
df.loc[0] = values 

['Jet Airways', 'Delhi', 'Cochin', 1, 24, 6, 13, 0, 4, 25, 15, 25]


In [38]:
columns.shape

AttributeError: 'list' object has no attribute 'shape'

In [39]:
import os
os.chdir(r "C:\Users\wasim\Office\Machine Learning Projects\Flight Fare Prediction ")

In [40]:
import dill
def load_object(file_path:str):
    """
    file_path: str
    """
    with open(file_path, "rb") as file_obj:
        return dill.load(file_obj)

In [41]:
model = load_object(file_path=r "C:\Users\wasim\Office\Machine Learning Projects\Flight Fare Prediction\flightFare\artifact\model_trainer\2022-09-10-00-38-22\trained_model\model.pkl ")

In [73]:
insurance_value = model.predict(df)

In [74]:
insurance_value

array([13618.2321952])